In [1]:
from magma import DeclareFromVerilog

source = '''\
module CSA_4 ( input [3:0] a,b,c, output [3:0] s, co);
   assign s = a ^ b ^c;
   assign co = a&b | b&c | a&c;
endmodule'''

CSA4 = DeclareFromVerilog(source)[0]

print(str(CSA4))
print(repr(CSA4))

CSA_4
CSA_4 = DeclareCircuit("CSA_4", "a", Array(4,In(Bit)), "b", Array(4,In(Bit)), "c", Array(4,In(Bit)), "s", Array(4,Out(Bit)), "co", Array(4,Out(Bit)))


In [2]:
from magma import DefineFromTemplatedVerilog

source = '''\
module CSA${N} ( input [${N-1}:0] a,b,c, output [${N-1}:0] s, co );
   assign s = a ^ b ^c;
   assign co = a&b | b&c | a&c;
endmodule'''

CSA4 = DefineFromTemplatedVerilog(source, **dict(N=4))[0]

In [3]:
from magma.verilator.function import testvectors
from magma.verilator.verilator import compile as compileverilator
from magma.verilator.verilator import run_verilator_test
import magma as m

def reference(a, b, c):
    s = a ^ b ^ c
    co = a & b | b & c | a & c
    return s, co

m.compile("build/csa4", CSA4)

compiling CSA4


In [4]:
with open("build/csa4.v", "r") as csa4_verilog:
    print(csa4_verilog.read())

module CSA4 ( input [3:0] a,b,c, output [3:0] s, co );
   assign s = a ^ b ^c;
   assign co = a&b | b&c | a&c;
endmodule



In [5]:
test_vectors = testvectors(CSA4, reference)
compileverilator('build/sim_test_csa_main.cpp', CSA4, test_vectors)
with open("build/sim_test_csa_main.cpp", "r") as verilator_harness:
    print(verilator_harness.read())

#include "VCSA4.h"
#include "verilated.h"
#include <cassert>
#include <iostream>

int main(int argc, char **argv, char **env) {
    Verilated::commandArgs(argc, argv);
    VCSA4* top = new VCSA4;

    unsigned int tests[4096][5] = {
        { 0b0000, 0b0000, 0b0000, 0b0000, 0b0000 }, 
        { 0b0000, 0b0000, 0b0001, 0b0001, 0b0000 }, 
        { 0b0000, 0b0000, 0b0010, 0b0010, 0b0000 }, 
        { 0b0000, 0b0000, 0b0011, 0b0011, 0b0000 }, 
        { 0b0000, 0b0000, 0b0100, 0b0100, 0b0000 }, 
        { 0b0000, 0b0000, 0b0101, 0b0101, 0b0000 }, 
        { 0b0000, 0b0000, 0b0110, 0b0110, 0b0000 }, 
        { 0b0000, 0b0000, 0b0111, 0b0111, 0b0000 }, 
        { 0b0000, 0b0000, 0b1000, 0b1000, 0b0000 }, 
        { 0b0000, 0b0000, 0b1001, 0b1001, 0b0000 }, 
        { 0b0000, 0b0000, 0b1010, 0b1010, 0b0000 }, 
        { 0b0000, 0b0000, 0b1011, 0b1011, 0b0000 }, 
        { 0b0000, 0b0000, 0b1100, 0b1100, 0b0000 }, 
        { 0b0000, 0b0000, 0b1101, 0b1101, 0b0000 }, 
        { 0b0000, 0b0000,

In [6]:
run_verilator_test('csa4', 'sim_test_csa_main', 'CSA4')  # causes an assertion if fails
print("Success")

Success
